# Using SEDkit with SIMPLE

This notebook will walk you through an example of how you can use the sedsimple.py package to use SEDkit with data loaded from the SIMPLE database.

## Requirements

You must have sedsimple.py in the current working directory (or in your python path) for imports to properly work. You will also need a copy of the SIMPLE database. This is most easily done with the binary file available at https://github.com/SIMPLE-AstroDB/SIMPLE-binary. You will also need to have SEDkit and AstrodbKit2 installed. See https://github.com/dr-rodriguez/SEDkitSIMPLE for more details.

In [1]:
from astropy.table import Table
from sedsimple import SEDSIMPLE
from astrodbkit2.astrodb import Database

## Initialization

The first step you need to do is to establish a connection to the SIMPLE database. If you have it in your current working directory it's as easy as:

In [2]:
db = Database('sqlite:///SIMPLE.db')

Refer to the AstrodbKit2 documentation at https://astrodbkit2.readthedocs.io/en/latest/ if you want more information on what you can do with the database. For now, we'll move into establishing an SED object that includes data from SIMPLE.

In [3]:
sed = SEDSIMPLE(db, 'Trappist-1', auto_db=True)

[sedkit] SED initialized
[sedkit] Setting name to 'Trappist-1'
[sedkit] 1 record for Trappist-1 found in Simbad.
[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (23.10815789, -5.04139917)>
[sedkit] There was a problem determining the interstellar reddening. Setting to 0. You can manually set this with the 'reddening' attribute.
[sedkit] Setting parallax to (<Quantity 80.2123 mas>, <Quantity 0.0716 mas>) with reference '2020yCat.1350....0G'
[sedkit] Setting distance to (<Quantity 12.47 pc>, <Quantity 0.01 pc>, <Quantity 0.01 pc>) with reference '2020yCat.1350....0G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] Setting radius to 'None'
[sedkit] Setting radius to (<Quantity 0.15 solRad>, <Quantity 0.02 solRad>, <Quantity 0.01 solRad>) with reference '2013ApJS..208....9P'
[sedkit] Setting spectral_type to (67.5, 0.5, 'V', None, None) with reference '2000AJ....120.1085G'
[SIMPLE] Trappist-1 matched to 2MASS J23062928-0502

The SEDSIMPLE Class inherits from SEDkit's SED Class and as such can do everything already implemented. It adds a few extra methods for loading data, which are automatically called if `auto_db=True` is set. 
You may notice above some messages with `[sedkit]` and `[SIMPLE]`. The former come from the original code, while the later come from this implementation. Let's walk through what happened.

## Walkthrough

The first thing that SEDSIMPLE does, regardless of whether you have `auto_db` set, is to check the provided name against the database. In this case, Trappist-1 is known as 2MASS J23062928-0502285 in the database. For convenience, this name and the inventory of data is stored in the SEDSIMPLE object:

In [4]:
print(sed.db_source)

2MASS J23062928-0502285


In [5]:
# Browse inventory; converting to Astropy Table for easier reading
for key, tab in sed.inventory.items():
    print('=========================')
    print(key)
    print(Table(tab))

Sources
         source             ra       dec    epoch equinox shortname reference other_references comments
----------------------- --------- --------- ----- ------- --------- --------- ---------------- --------
2MASS J23062928-0502285 346.62201 -5.041274  None    None 2306-0502    Gizi00             None     None
Names
          other_name         
-----------------------------
      2MASS J23062928-0502285
 Gaia DR2 2635476908753563008
Gaia EDR3 2635476908753563008
                   Trappist-1
Parallaxes
     parallax        parallax_error   adopted comments reference
----------------- ------------------- ------- -------- ---------
80.45124261077767 0.12107673660785581   False     None   GaiaDR2
80.21231612635748 0.07160574197769165    True     None  GaiaEDR3
Photometry
   band     ucd        magnitude         magnitude_error    telescope instrument epoch comments reference
--------- -------- ------------------ --------------------- --------- ---------- ----- -------- ---------


At any point, you can refer to the data by accessing the inventory and providing the table name. The default format is a list of dictionary entries.

In [6]:
print(sed.inventory['SpectralTypes'])

[{'spectral_type_string': 'M8', 'spectral_type_code': 68.0, 'spectral_type_error': None, 'regime': 'nir_UCD', 'adopted': None, 'comments': None, 'reference': 'Gill16'}, {'spectral_type_string': 'M7.5', 'spectral_type_code': 67.5, 'spectral_type_error': None, 'regime': 'optical', 'adopted': None, 'comments': None, 'reference': 'Gizi00'}]


When `auto_db` is set, SEDSIMPLE calls several methods that each load specific information. We'll call them again one by one here.

First, we'll load coordinates, which uses information from the Sources table:

In [7]:
sed.load_coords_db()

[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (346.62201, -5.041274)>
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'


Next, we'll load distance information from the Parallaxes table:

In [8]:
sed.load_parallax_db()

[sedkit] Setting parallax to (<Quantity 80.21231613 mas>, <Quantity 0.07160574 mas>) with reference '2021A&A...649A...1G'
[sedkit] Setting distance to (<Quantity 12.47 pc>, <Quantity 0.01 pc>, <Quantity 0.01 pc>) with reference '2021A&A...649A...1G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'


Next, we'll load any photometry from the Photometry table:

In [9]:
sed.load_photometry_db()

[sedkit] Setting 2MASS.H photometry to 10.718 (0.021) with reference '2003tmc..book.....C'
[sedkit] Setting 2MASS.J photometry to 11.354 (0.022) with reference '2003tmc..book.....C'
[sedkit] Setting 2MASS.Ks photometry to 10.296 (0.023) with reference '2003tmc..book.....C'
[sedkit] Setting Gaia.G photometry to 15.645 (0.001) with reference '2018A&A...616A...1G'
[sedkit] Setting Gaia.rp photometry to 14.097 (0.009) with reference '2018A&A...616A...1G'
[sedkit] Setting Gaia.G photometry to 15.623 (0.001) with reference '2021A&A...649A...1G'
[sedkit] Setting Gaia.rp photometry to 14.104 (0.002) with reference '2021A&A...649A...1G'
[sedkit] Setting WISE.W1 photometry to 10.042 (0.023) with reference '2012yCat.2311....0C'
[sedkit] Setting WISE.W2 photometry to 9.799 (0.020) with reference '2012yCat.2311....0C'
[sedkit] Setting WISE.W3 photometry to 9.528 (0.041) with reference '2012yCat.2311....0C'
[sedkit] Setting WISE.W4 photometry to 8.397 (nan) with reference '2012yCat.2311....0C'


Next, we'll load the best (adopted) spectral type from the SpectralTypes table:

In [10]:
sed.load_spectral_type_db()

Note that there was no output. This is because no spectral type is marked as adopted for this particular source in the database. You can manually edit the database if you feel this is incorrect.

Finally, let's add all spectra from the Spectra table:

In [11]:
sed.load_spectra_db(column='spectrum')

[sedkit] Looks like that 0.6533203125 um-2.56640625 um spectrum is already added. Skipping...
Error loading https://s3.amazonaws.com/bdnyc/RCspec/U12171.fits: self input argument 'x' of shape (1479,) cannot be broadcast with parameter 'slope' of shape (3,).
[SIMPLE] Unable to parse spectrum from database 
{'spectrum': 'https://s3.amazonaws.com/bdnyc/RCspec/U12171.fits', 'original_spectrum': None, 'local_spectrum': '$BDNYC_spectra/RCspec/U12171.fits', 'regime': 'optical', 'telescope': 'KPNO 4m', 'instrument': 'R-C Spec', 'mode': None, 'observation_date': datetime.datetime(2003, 7, 10, 0, 0), 'wavelength_units': 'A', 'flux_units': 'erg s-1 cm-2 A-1', 'wavelength_order': None, 'comments': None, 'reference': 'Cruz07', 'other_references': None}
Error: 'numpy.str_' object has no attribute 'wavelength'


There are two available spectra for this source, but only one of them can be easily imported. The other fails due to the formatting of the data and will need be manually transformed. This is beyond the scope of this tutorial. 

Note the use of `column='spectrum'`, this allows you to load data from different fields in the Spectra table. Most likely you will want spectrum, but original_spectrum and local_spectrum are other options.

If you want to fetch an individual spectrum from the database, perhaps to do your own modifications you can use:

In [12]:
spec = sed.inventory['Spectra'][0]  # select first entry in the Spectra table
wave, flux, flux_unc, ref = sed.fetch_single_spectrum_db(spec)
print(len(wave), type(wave))
print(len(flux), type(flux))
print(len(flux_unc), type(flux_unc))
print(ref)

564 <class 'specutils.spectra.spectral_axis.SpectralAxis'>
564 <class 'astropy.units.quantity.Quantity'>
564 <class 'astropy.units.quantity.Quantity'>
2018AJ....155...34C


### !! As a reminder: you do not need to call these individual methods if you are using `auto_db=True` !!

## Generating the SED

With these data in place, either loaded manually or by `auto_db` you can now proceed to the usual steps in SED generation.

In [13]:
sed.plot()

[sedkit] Setting fbol to (<Quantity 1.24153219e-10 erg / (cm2 s)>, <Quantity 3.79366668e-13 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 2.3099461e+30 erg / s>, <Quantity 7.97156513e+27 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-3.219, 0.001) with reference 'This Work'
[sedkit] Setting mbol to (13.283, 0.003) with reference 'This Work'
[sedkit] Setting Mbol to (12.804, 0.003) with reference 'This Work'
[sedkit] Setting logg to 'None'
[sedkit] Could not calculate logg without Lbol and age
[sedkit] Setting mass to 'None'
[sedkit] Setting mass to (<Quantity 0.07 solMass>, <Quantity 0. solMass>, <Quantity 0. solMass>) with reference '2013ApJS..208....9P'
[sedkit] Setting Teff to (<Quantity 2335 K>, <Quantity 155 K>) with reference 'This Work'


Figure(id='1033', ...)

In [14]:
sed.results

param,value,unc,units
object,object,object,object
name,Trappist-1,--,--
ra,346.62201,--,deg
dec,-5.041274,--,deg
age,--,--,--
membership,--,--,--
distance,12.47,0.01,pc
parallax,80.21231612635748,0.07160574197769165,mas
SpT,M7.5V,--,--
spectral_type,67.5,0.5,--
